In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
import nibabel as nib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
from glob import glob
from tqdm import tqdm
import pandas as pd
import os
import pydicom
import nibabel as nib
import numpy as np
import pandas as pd
import scipy.ndimage
from glob import glob
from tqdm import tqdm
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:


data_dir = '../../data/250513 C CT/'
save_dir = '../../data/ct_nii/'
df = pd.read_csv('../../data/filter_ct_df.csv')

def window_and_normalize(img, dcm):
    # HU 변환
    slope = dcm.RescaleSlope if 'RescaleSlope' in dcm else 1
    intercept = dcm.RescaleIntercept if 'RescaleIntercept' in dcm else 0
    img = img * slope + intercept

    # WL/WW 적용
    wl = dcm.WindowCenter[0] if isinstance(dcm.WindowCenter, pydicom.multival.MultiValue) else dcm.WindowCenter
    ww = dcm.WindowWidth[0] if isinstance(dcm.WindowWidth, pydicom.multival.MultiValue) else dcm.WindowWidth

    if wl is not None and ww is not None:
        min_val = wl - ww / 2
        max_val = wl + ww / 2
        img = np.clip(img, min_val, max_val)
        img = (img - min_val) / (max_val - min_val + 1e-8)
    else:
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)

    # [-1, 1] 스케일링
    img = img * 2 - 1
    return img

folder_list = [os.path.join(data_dir, str(pid)) for pid in df['ID']]

for j in tqdm(range(len(folder_list))):
    dcm_list = sorted(glob(f'{folder_list[j]}/*.dcm'))
    if not dcm_list:
        continue

    volume_img = []
    for dcm_file in dcm_list:
        dcm = pydicom.dcmread(dcm_file)
        img = dcm.pixel_array
        img_norm = window_and_normalize(img, dcm)

        # 0~255로 변환 후 다시 float32로 [-1, 1] 재정규화
        gray = (img_norm * 127.5 + 127.5).astype(np.uint8)
        volume_img.append(gray.astype(np.float32) / 127.5 - 1)

    volume_img = np.stack(volume_img, axis=0)  # (Z, H, W)

    # ✅ 리샘플: SliceThickness는 유지, PixelSpacing만 0.5mm로 고정
    sample_dcm = pydicom.dcmread(dcm_list[0])
    try:
        y_spacing, x_spacing = map(float, sample_dcm.PixelSpacing)
    except:
        y_spacing, x_spacing = 1.0, 1.0  # fallback
    zoom_factors = [1.0, y_spacing / 0.5, x_spacing / 0.5]  # (Z, Y, X)

    volume_resampled = scipy.ndimage.zoom(volume_img, zoom=zoom_factors, order=1)  # trilinear

    # 저장
    nii_name = os.path.basename(folder_list[j])
    os.makedirs(save_dir, exist_ok=True)
    nii_img = nib.Nifti1Image(volume_resampled.astype(np.float32), affine=np.eye(4))
    nib.save(nii_img, os.path.join(save_dir, f'{nii_name}.nii.gz'))
